In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
#import graph, coarsening, utils
%load_ext autoreload
%autoreload 1
%aimport graph
%aimport coarsening
%aimport utils

import tensorflow as tf
import time, shutil
import numpy as np
import os, collections, sklearn
import scipy.sparse as sp
import matplotlib.pyplot as plt
import networkx as nx
tf.logging.set_verbosity(tf.logging.ERROR)
%matplotlib inline

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
#Definition of some flags useful later in the code

number_edges = 8
metric = 'euclidean'
normalized_laplacian = True
coarsening_levels = 4
# Directories.
dir_data = 'data_mnist'

In [3]:
#Here we proceed at computing the original grid where the images live and the various coarsening that are applied
#for each level

def grid_graph(m): # Create the graph as a grid
    z = graph.grid(m)  # normalized nodes coordinates
    dist, idx_ = graph.distance_sklearn_metrics(z, k=number_edges, metric=metric) 
    #dist contains the distance of the 8 nearest neighbors for each node indicated in z sorted in ascending order
    #idx contains the indexes of the 8 nearest for each node sorted in ascending order by distance

    A_ = graph.adjacency(dist, idx_)  # graph.adjacency() builds a sparse matrix out of the identified edges computing similarities as: A_{ij} = e^(-dist_{ij}^2/sigma^2)
    
    return A_, z

def coarsen_for_MoNet(A, nodes_coordinates, levels):
    graphs, parents = coarsening.metis(A, levels) #Coarsen a graph multiple times using Graclus variation of the METIS algorithm. 
                                                  #Basically, we randomly sort the nodes, we iterate on them and we decided to group each node
                                                  #with the neighbor having highest w_ij * 1/(\sum_k w_ik) + w_ij * 1/(\sum_k w_kj) 
                                                  #i.e. highest sum of probabilities to randomly walk from i to j and from j to i.
                                                  #We thus favour strong connections (i.e. the ones with high weight wrt all the others for both nodes) 
                                                  #in the choice of the neighbor of each node.
                    
                                                  #Construction is done a priori, so we have one graph for all the samples!
                    
                                                  #graphs = list of spare adjacency matrices (it contains in position 
                                                  #          0 the original graph)
                                                  #parents = list of numpy arrays (every array in position i contains 
                                                  #           the mapping from graph i to graph i+1, i.e. the idx of
                                                  #           node i in the coarsed graph -> that is, the idx of its cluster) 
    perms = coarsening.compute_perm(parents) #Return a list of indices to reorder the adjacency and data matrices so
                                             #that two consecutive nodes correspond to neighbors that should be collapsed
                                             #to produce the coarsed version of the graph.
                                             #Fake nodes are appended for each node which is not grouped with anybody else
    coordinates = np.copy(nodes_coordinates)
    
    u_rows, u_cols, u_values, u_shapes = [], [], [], []
    for i,A in enumerate(graphs):
        M = A.shape[0]

        # We remove self-connections created by metis.
        A = A.tocoo()
        A.setdiag(0)

        if i < levels: #if we have to pool the graph 
            A = coarsening.perm_adjacency(A, perms[i]) #matrix A is here extended with the fakes nodes
                                                       #in order to do an efficient pooling operation
                                                       #in tensorflow as it was a 1D pooling
        A = A.tocsr()
        A.eliminate_zeros()
        
        Mnew = A.shape[0]
        u_shapes.append(Mnew)          
        if i == 0:
            # since we use distances the coordinate vec needs to be extended to the fakes node
            number_of_fake_node = Mnew-M
            coordinates = np.concatenate([coordinates, np.ones([number_of_fake_node, 2])*np.inf], 0)
            coordinates = coordinates[perms[0]]       
        
        print('Layer {0}: U_{0} = |V| = {1} nodes ({2} added), |E| = {3} edges'.format(i, Mnew, Mnew-M, A.nnz//2))

        from_node, to_node = A.nonzero()
        u_rows.append(from_node); u_cols.append(to_node)
        
        dist_nodes_vec = coordinates[from_node] - coordinates[to_node]
        u_values.append(dist_nodes_vec)
        
        
        # update coordinates for next coarser graph
        new_coordinates = []
        for k in range(A.shape[0]//2):
            idx_first_el = k * 2
            
            if not np.isfinite(coordinates[idx_first_el][0]):
                new_coordinates.append(coordinates[idx_first_el+1])
                
            elif not np.isfinite(coordinates[idx_first_el+1][0]):
                new_coordinates.append(coordinates[idx_first_el])
                
            else:
                new_coordinates.append(np.mean(coordinates[idx_first_el:idx_first_el+2], axis=0))
                
        coordinates = np.asarray(new_coordinates)
        
    U = []
    for it in range(coarsening_levels):
        U.append([u_rows[it], u_cols[it], u_values[it], u_shapes[it]])
    print("shape_u 0:"+str(len(u_values[0])))
    return  U, perms[0]


In [4]:
def load_mnist():
    #loading of MNIST dataset
    from tensorflow.examples.tutorials.mnist import input_data
    mnist_ = input_data.read_data_sets(dir_data, one_hot=False)
    train_data_ = mnist_.train.images.astype(np.float32)
    val_data_ = mnist_.validation.images.astype(np.float32) #the first 5K samples of the training dataset 
    #are used for validation
    test_data_ = mnist_.test.images.astype(np.float32)
    train_labels_ = mnist_.train.labels
    val_labels_ = mnist_.validation.labels
    test_labels_ = mnist_.test.labels
    t_start = time.time()
    np.random.seed(0)
    n_rows_cols = 28
    A_, nodes_coordinates = grid_graph(n_rows_cols)
    U, perm_OG = coarsen_for_MoNet(A_, nodes_coordinates, coarsening_levels)
    print('Execution time: {:.2f}s'.format(time.time() - t_start))
   
    train_data_ = coarsening.perm_data(train_data_, perm_OG)
    val_data_ = coarsening.perm_data(val_data_, perm_OG)
    test_data_ = coarsening.perm_data(test_data_, perm_OG)
    del perm_OG
    return  U, mnist_, test_data_, test_labels_, train_data_, train_labels_, val_data_, val_labels_, A_



In [5]:
# class ChebNet:
class MyMoNet:
    """
    The neural network model.
    """
    
    #Helper functions used for constructing the model
    def _weight_variable(self, shape, regularization=True): 
        """Initializer for the weights"""
        
        initial = tf.truncated_normal_initializer(0, 0.1)
        var = tf.get_variable('weights', shape, tf.float32, initializer=initial)
        if regularization: #append the loss of the current variable to the regularization term 
            self.regularizers.append(tf.nn.l2_loss(var))
        return var
    
    def _bias_variable(self, shape, regularization=True):
        """Initializer for the bias"""
        
        initial = tf.constant_initializer(0.1)
        var = tf.get_variable('bias', shape, tf.float32, initializer=initial)
        if regularization:
            self.regularizers.append(tf.nn.l2_loss(var))
        return var
    

    def frobenius_norm(self, tensor): 
        """Computes the frobenius norm for a given tensor"""
        
        square_tensor = tf.square(tensor)
        tensor_sum = tf.reduce_sum(square_tensor)
        frobenius_norm = tf.sqrt(tensor_sum)
        return frobenius_norm
    
    
    def count_no_weights(self):
        total_parameters = 0
        for variable in tf.trainable_variables():
            # shape is an array of tf.Dimension
            shape = variable.get_shape()
            variable_parameters = 1
            for dim in shape:
                variable_parameters *= dim.value
            total_parameters += variable_parameters
        print('#weights in the model: %d' % (total_parameters,))

    def patch_operator(self,j,u, x): # D_j(x)f patch operator
        N, M, Fin = x.get_shape()
        indices_row_, indices_col_, values_u_, shape_u_ = u
        with tf.variable_scope('guassian_filter_{}'.format(j)):  #aka the kernel
            mu_j = tf.get_variable('mu_{}'.format(j), [1, values_u_.shape[1]], tf.float32)
            precision_j = tf.get_variable('precision_{}'.format(j), [1, values_u_.shape[1]], tf.float32, initializer=tf.initializers.random_uniform(minval=0.1,maxval=1))
            
            j_ = tf.reduce_sum((-0.5)*(values_u_-mu_j)*precision_j*(values_u_-mu_j), axis=-1)
   
            w_j=  tf.exp(j_, name='w_{}'.format(j))
            w_gaussian = tf.SparseTensor(indices=np.vstack([indices_row_, indices_col_]).T, 
                                        values=w_j, 
                                        dense_shape=[shape_u_,shape_u_] )
        x_t = tf.reshape(tf.transpose(x, [1,2,0]), [M, Fin*N])  
        temp__=tf.sparse_tensor_dense_matmul(w_gaussian, x_t)
        
        return tf.transpose(tf.reshape(temp__, [M, Fin, N]), [2,0,1])
    
    
    def MyMoNetConv(self, x, u, Fout, n_filters): 
        """Applies chebyshev polynomials over the graph (i.e. it makes a spectral convolution)"""
        N, M, Fin = x.get_shape()  # N is the number of images # batch size 
                                   # M the number of vertices in the images
                                   # Fin the number of features
        d_j_list=[x]
        for j in range(n_filters): # Cycle through the Gaussian
            d_j = self.patch_operator(j,u,x)
            d_j_list.append(d_j)
        d_j_conv=tf.stack(d_j_list,0)
        
        d_j_conv = tf.transpose(d_j_conv, [1,2,3,0]) 
        d_j_conv = tf.reshape(d_j_conv, [N*M, Fin*(n_filters+1)]) 
        G = self._weight_variable([Fin*(n_filters+1), Fout])
        out_conv = tf.matmul(d_j_conv, G) 
        result = tf.reshape(out_conv, [N, M, Fout])
        return result
    
    
    def b1relu(self, x):
        """Applies bias and ReLU. One bias per filter."""
        N, M, F = x.get_shape()
        b = self._bias_variable([1, 1, int(F)], regularization=False)
        return tf.nn.relu(x + b) #add the bias to the convolutive layer


    def mpool1(self, x, p):
        """Max pooling of size p. Should be a power of 2 (this is possible thanks to the reordering we previously did)."""
        if p > 1:
            x = tf.expand_dims(x, 3)  # shape = N x M x F x 1
            x = tf.nn.max_pool(x, ksize=[1,p,1,1], strides=[1,p,1,1], padding='SAME')
            return tf.squeeze(x, [3])  # shape = N x M/p x F
        else:
            return x

    def fc(self, x, Mout, relu=True):
        """Fully connected layer with Mout features."""
        N, Min = x.get_shape()
        W = self._weight_variable([int(Min), Mout], regularization=True)
        b = self._bias_variable([Mout], regularization=True)
        x = tf.matmul(x, W) + b
        return tf.nn.relu(x) if relu else x
    
    def def_layer_stack(self, i, x):
        with tf.variable_scope('layer_stack_{}'.format(i)):
            with tf.name_scope('Monet_conv'):
                conv_out = self.MyMoNetConv(x,self.u[i*2],self.F[i], self.K[i])
            with tf.name_scope('relu'):
                relu_out = self.b1relu(conv_out)
            with tf.name_scope('pool'):
                pool_out = self.mpool1(relu_out, self.p[i])
        return pool_out
    
    #function used for extracting the result of our model
    def _inference(self, x, dropout): #definition of the model
       # Graph convolutional layers.
        x = tf.expand_dims(x, 2)  # N x M x F=1
        for i in range(self.num_layers):
            x = self.def_layer_stack(i,x)
        pool_out=x
        # Fully connected hidden layers.
        N, M, F = pool_out.get_shape()
        pool_out_reshaped = tf.reshape(pool_out, [int(N), int(M*F)])  # N x M
        for i,M in enumerate(self.M[:-1]): #apply a fully connected layer for each layer defined in M
                                           #(we discard the last value in M since it contains the number of classes we have
                                           #to predict)
            with tf.variable_scope('fc{}'.format(i+1)):
                fc_out = self.fc(pool_out_reshaped, M)
                dropout_out = tf.nn.dropout(fc_out, dropout)
        
        # Logits linear layer, i.e. softmax without normalization.
        with tf.variable_scope('logits'):
            logits = self.fc(dropout_out, self.M[-1], relu=False)
        return logits
    
    def convert_coo_to_sparse_tensor(self, L):
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data.astype('float32'), L.shape)
        L = tf.sparse_reorder(L)
        return L
        
    
    def __init__(self, p, K, F, M, M_0, batch_size, u ,decay_steps, decay_rate, learning_rate=1e-4, momentum=0.9, regularization=5e-4, num_layers= 2,idx_gpu = '/gpu:0'):
        self.regularizers = list()  # list of regularization l2 loss for multiple variables
        self.num_layers = num_layers
        self.p = p                  # dimensions of the pooling layers
        self.K = K                  # Number of Gaussian used 
        self.dim_d = 2              # dimensionality of function u(x,y) with x vertex and y vertex in N(x)
        self.F = F                  # Number of features of convolutional layers
        self.M = M                  # Number of neurons in fully connected layers
        self.M_0 = M_0              # number of elements in the first graph 
        self.batch_size = batch_size 
        
                        #definition of some learning parameters
        self.decay_steps = decay_steps
        self.decay_rate = decay_rate
        self.learning_rate = learning_rate
        self.regularization = regularization 
        with tf.Graph().as_default() as g:
                self.graph = g
                tf.set_random_seed(0)
                with tf.device(idx_gpu):
                        self.u = u
                        #definition of placeholders
                        self.ph_data = tf.placeholder(tf.float32, (self.batch_size, M_0), 'data')
                        self.ph_labels = tf.placeholder(tf.int32, self.batch_size, 'labels')
                        self.ph_dropout = tf.placeholder(tf.float32, (), 'dropout')
                    
                        #Model construction
                        self.logits = self._inference(self.ph_data, self.ph_dropout)
                        
                        #Definition of the loss function
                        with tf.name_scope('loss'):
                            print(self.logits)
                            print(self.ph_labels)
                            self.cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=self.logits, 
                                                                                                labels=self.ph_labels)
                            self.cross_entropy = tf.reduce_mean(self.cross_entropy)
                        with tf.name_scope('regularization'):
                            self.regularization *= tf.add_n(self.regularizers)
                        self.loss = self.cross_entropy #+ self.regularization
                        
                        #Solver Definition
                        with tf.name_scope('training'):
                            # Learning rate.
                            global_step = tf.Variable(0, name='global_step', trainable=False) #used for counting how many iterations we have done
                            if decay_rate != 1: #applies an exponential decay of the lr wrt the number of iterations done
                                learning_rate = tf.train.exponential_decay(
                                        learning_rate, global_step, decay_steps, decay_rate, staircase=True)
                            # Optimizer.
                            # if momentum == 0:
                            #     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
                            #  else: #applies momentum for increasing the robustness of the gradient 
                            #     optimizer = tf.train.MomentumOptimizer(learning_rate, momentum)
                            # grads = optimizer.compute_gradients(self.loss)
                            # self.op_gradients = optimizer.apply_gradients(grads, global_step=global_step)
                            optimizer = tf.train.RMSPropOptimizer(learning_rate, momentum=0.8)
                            self.op_gradients = optimizer.minimize(self.loss)
                            
                        #Computation of the norm gradients (useful for debugging)
                        self.var_grad = tf.gradients(self.loss, tf.trainable_variables())
                        self.norm_grad = self.frobenius_norm(tf.concat([tf.reshape(g, [-1]) for g in self.var_grad if g!=None], 0))

                        #Extraction of the predictions and computation of accuracy
                        self.predictions = tf.cast(tf.argmax(self.logits, dimension=1), tf.int32)
                        self.accuracy = 100 * tf.contrib.metrics.accuracy(self.predictions, self.ph_labels)
        
                        # Create a session for running Ops on the Graph.
                        config = tf.ConfigProto(allow_soft_placement = True)
                        config.gpu_options.allow_growth = True
                        self.session = tf.Session(config=config)

                        # Run the Op to initialize the variables.
                        init = tf.global_variables_initializer()
                        self.session.run(init)
                        
                        self.count_no_weights()

In [6]:
def run():
    U, mnist, test_data, test_labels, train_data, train_labels, val_data, val_labels, A = load_mnist()
    np.random.seed(0)
    #Convolutional parameters
    p = [4, 4]   #Dimensions of the pooling layers
    K = [25, 25] #List of polynomial orders, i.e. filter sizes or number of hops
    F = [32, 64] #Number of features of convolutional layers
    
    #FC parameters
    C = max(mnist.train.labels) + 1 #Number of classes we have
    M = [512, C] #Number of neurons in fully connected layers
    
    #Solver parameters
    batch_size = 100
    decay_steps = mnist.train.num_examples / batch_size #number of steps to do before decreasing the learning rate
    decay_rate = 0.95 #how much decreasing the learning rate
    learning_rate = 1e-5
    momentum = 0.9
    regularization = 1e-4
    
    #Definition of keep probabilities for dropout layers
    dropout_training = 0.5
    dropout_val_test = 1.0
    #%%
  
    M_0 = U[0][3] #number of elements in the first graph
    print("M_0: "+str(M_0))
    learning_obj = MyMoNet(p, K, F, M, M_0, batch_size, U, decay_steps, decay_rate, learning_rate=learning_rate, regularization=regularization,momentum=momentum)
    # learning_obj = MyMoNet(p, K, F, M, M_0, batch_size, L,u ,decay_steps, decay_rate, learning_rate=1e-4, momentum=0.9, regularization=5e-4, num_layers= 2,idx_gpu = '/gpu:0')
    
    #definition of overall number of training iterations and validation frequency
    num_iter_val = 100
    num_total_iter_training = 2100
    
    num_iter = 0
    
    list_training_loss = list()
    list_training_norm_grad = list()
    list_val_accuracy = list()
    #%%
    #training and validation
    indices = collections.deque() #queue that will contain a permutation of the training indexes
    for k in range(num_iter, num_total_iter_training):
        
        #Construction of the training batch
        if len(indices) < batch_size: # Be sure to have used all the samples before using one a second time.
            indices.extend(np.random.permutation(train_data.shape[0])) #reinitialize the queue of indices
        idx = [indices.popleft() for i in range(batch_size)] #extract the current batch of samples
        
        #data extraction
        batch_data, batch_labels = train_data[idx,:], train_labels[idx] 
        
        feed_dict = {learning_obj.ph_data: batch_data, 
                     learning_obj.ph_labels: batch_labels, 
                     learning_obj.ph_dropout: dropout_training}
        
        #Training
        tic = time.time()
        _, current_training_loss, norm_grad, logii = learning_obj.session.run([learning_obj.op_gradients, 
                                                                        learning_obj.loss, 
                                                                        learning_obj.norm_grad, learning_obj.logits], feed_dict = feed_dict) 
        training_time = time.time() - tic
        
        list_training_loss.append(current_training_loss)
        list_training_norm_grad.append(norm_grad)
        #print(np.any(np.isnan(logii)))
        if (np.mod(num_iter, num_iter_val)==0): #validation
            msg = "[TRN] iter = %03i, cost = %3.2e, |grad| = %.2e (%3.2es)" \
                        % (num_iter, list_training_loss[-1], list_training_norm_grad[-1], training_time)
            print(msg)
            
            #Validation Code
            tic = time.time()
            val_accuracy = 0
            for begin in range(0, val_data.shape[0], batch_size):
                end = begin + batch_size
                end = min([end, val_data.shape[0]])
                
                #data extraction
                batch_data = np.zeros((end-begin, val_data.shape[1]))
                batch_data = val_data[begin:end,:]
                batch_labels = np.zeros(batch_size)
                batch_labels[:end-begin] = val_labels[begin:end]
                
                feed_dict = {learning_obj.ph_data: batch_data, 
                             learning_obj.ph_labels: batch_labels,
                             learning_obj.ph_dropout: dropout_val_test}
                
                batch_accuracy = learning_obj.session.run(learning_obj.accuracy, feed_dict)
                val_accuracy += batch_accuracy*batch_data.shape[0]
            val_accuracy = val_accuracy/val_data.shape[0]
            val_time = time.time() - tic
            msg = "[VAL] iter = %03i, acc = %4.2f (%3.2es)" % (num_iter, val_accuracy, val_time)
            print(msg)
        num_iter += 1
    #Test code
    tic = time.time()
    test_accuracy = 0
    for begin in range(0, test_data.shape[0], batch_size):
        end = begin + batch_size
        end = min([end, test_data.shape[0]])
                
        batch_data = np.zeros((end-begin, test_data.shape[1]))
        batch_data = test_data[begin:end,:]
                
        feed_dict = {learning_obj.ph_data: batch_data, learning_obj.ph_dropout: 1}
                
        batch_labels = np.zeros(batch_size)
        batch_labels[:end-begin] = test_labels[begin:end]
        feed_dict[learning_obj.ph_labels] = batch_labels
                
        batch_accuracy = learning_obj.session.run(learning_obj.accuracy, feed_dict)
        test_accuracy += batch_accuracy*batch_data.shape[0]
    test_accuracy = test_accuracy/test_data.shape[0]
    test_time = time.time() - tic
    msg = "[TST] iter = %03i, acc = %4.2f (%3.2es)" % (num_iter, test_accuracy, test_time)
    print(msg)

In [7]:
run()

Extracting data_mnist/train-images-idx3-ubyte.gz
Extracting data_mnist/train-labels-idx1-ubyte.gz
Extracting data_mnist/t10k-images-idx3-ubyte.gz
Extracting data_mnist/t10k-labels-idx1-ubyte.gz
Layer 0: U_0 = |V| = 944 nodes (160 added), |E| = 3198 edges
Layer 1: U_1 = |V| = 472 nodes (64 added), |E| = 1426 edges
Layer 2: U_2 = |V| = 236 nodes (22 added), |E| = 653 edges
Layer 3: U_3 = |V| = 118 nodes (6 added), |E| = 318 edges
Layer 4: U_4 = |V| = 59 nodes (0 added), |E| = 152 edges
shape_u 0:6396
Execution time: 0.37s
M_0: 944
Tensor("logits/add:0", shape=(100, 10), dtype=float32, device=/device:GPU:0)
Tensor("labels:0", shape=(100,), dtype=int32, device=/device:GPU:0)
#weights in the model: 1993330
[TRN] iter = 000, cost = 8.64e+01, |grad| = 5.83e+02 (3.70e+00s)
[VAL] iter = 000, acc = 8.40 (1.02e+00s)
[TRN] iter = 100, cost = 9.84e+00, |grad| = 9.87e+01 (7.07e-02s)
[VAL] iter = 100, acc = 53.64 (7.35e-01s)
[TRN] iter = 200, cost = 1.47e+00, |grad| = 3.96e+01 (7.07e-02s)
[VAL] iter 